In [6]:
import pandas as pd
import json


# Wczytanie pliku z pominięciem pierwszego wiersza
glucose_data = pd.read_csv(
    'Michał._glucose_8-3-2025.csv',
    skiprows=1,               # Pomijamy pierwszy wiersz z metadanymi
    delimiter=',',
    low_memory=False
)

# Sprawdzenie nazw kolumn
print(glucose_data.columns)

# Konwersja daty na format datetime
glucose_data['Device Timestamp'] = pd.to_datetime(glucose_data['Device Timestamp'], errors='coerce')

# Wybór kluczowych kolumn
glucose_data = glucose_data[['Device Timestamp', 'Historic Glucose mg/dL', 'Scan Glucose mg/dL']]
glucose_data = glucose_data.rename(columns={
    'Device Timestamp': 'date',
    'Historic Glucose mg/dL': 'historic_glucose',
    'Scan Glucose mg/dL': 'scan_glucose'
})

# Podgląd danych
print(glucose_data.head())


# # Wczytanie pliku CSV z pomiarami glukozy
# glucose_data = pd.read_csv('Michał._glucose_8-3-2025.csv')
# print(glucose_data.head())

# # Wczytanie pliku JSON z informacjami o posiłkach
# with open('data (6).json', 'r') as f:
#     meals_data = json.load(f)

# # Przykładowe sprawdzenie struktury pliku JSON
# print(json.dumps(meals_data, indent=4))



Index(['Device', 'Serial Number', 'Device Timestamp', 'Record Type',
       'Historic Glucose mg/dL', 'Scan Glucose mg/dL',
       'Non-numeric Rapid-Acting Insulin', 'Rapid-Acting Insulin (units)',
       'Non-numeric Food', 'Carbohydrates (grams)', 'Carbohydrates (servings)',
       'Non-numeric Long-Acting Insulin', 'Long-Acting Insulin Value (units)',
       'Notes', 'Strip Glucose mg/dL', 'Ketone mmol/L', 'Meal Insulin (units)',
       'Correction Insulin (units)', 'User Change Insulin (units)'],
      dtype='object')
                 date  historic_glucose  scan_glucose
0 2024-09-17 18:12:00              92.0           NaN
1 2024-09-17 18:27:00              92.0           NaN
2 2024-09-17 18:42:00             111.0           NaN
3 2024-09-17 18:57:00             133.0           NaN
4 2024-09-17 19:12:00             140.0           NaN


C:\Users\Micha\AppData\Local\Temp\ipykernel_24748\708567084.py:17: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  glucose_data['Device Timestamp'] = pd.to_datetime(glucose_data['Device Timestamp'], errors='coerce')


In [45]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from sklearn.preprocessing import MinMaxScaler
import json

# ==== Wczytywanie pliku CSV ====
glucose_data = pd.read_csv(
    'Michał._glucose_8-3-2025.csv',
    skiprows=1,
    delimiter=',',
    low_memory=False
)

# Konwersja daty na format datetime
glucose_data['Device Timestamp'] = pd.to_datetime(
    glucose_data['Device Timestamp'], 
    format='%d-%m-%Y %H:%M', 
    errors='coerce'
)

# Usuwamy błędne daty
glucose_data = glucose_data.dropna(subset=['Device Timestamp'])

# ✅ Łączymy dane z dwóch kolumn (Historic + Scan)
glucose_data['glucose'] = glucose_data['Historic Glucose mg/dL'].combine_first(
    glucose_data['Scan Glucose mg/dL']
)

glucose_data = glucose_data.rename(columns={'Device Timestamp': 'date'})
glucose_data = glucose_data[['date', 'glucose']]

# ==== Wczytywanie pliku JSON ====
with open('data (6).json', 'r') as f:
    meals_data = json.load(f)

# Przetwarzanie danych JSON
processed_data = []
for user_id, records in meals_data.items():
    for record in records:
        calculator = record.get('calculatorData')
        if calculator:
            glucose = calculator.get('glucose')
            insulin = calculator.get('units', {}).get('short')
            date = pd.to_datetime(calculator.get('date'), errors='coerce')

            carbs = sum(meal.get('carbs', 0) for meal in record.get('meals', []))

            processed_data.append([date, glucose, insulin, carbs])

# Tworzymy DataFrame z JSON
meals_df = pd.DataFrame(processed_data, columns=['date', 'glucose', 'insulin', 'carbs'])
meals_df['date'] = meals_df['date'].dt.tz_convert(None)

# ==== Sortowanie danych ====
glucose_data = glucose_data.sort_values('date').reset_index(drop=True)
meals_df = meals_df.sort_values('date').reset_index(drop=True)

# ==== Dopasowywanie danych z glukometru ====
merged_data = []
for index, row in meals_df.iterrows():
    meal_time = row['date']
    
    glucose_window = glucose_data[
        (glucose_data['date'] >= meal_time) & 
        (glucose_data['date'] <= meal_time + pd.Timedelta(hours=3))
    ]

    if not glucose_window.empty:
        glucose_values = glucose_window['glucose'].values.tolist()
        glucose_max = np.nanmax(glucose_values)
        glucose_min = np.nanmin(glucose_values)

        dose_feedback = 0
        if glucose_max > 180:
            dose_feedback = 1
        elif glucose_min < 80:
            dose_feedback = -1

        WW = row['carbs'] / 10  
        insulin_per_WW = row['insulin'] / WW if WW > 0 else 0

        hour = meal_time.hour + meal_time.minute / 60
        day_of_week = meal_time.dayofweek / 6  

        total_dose = WW * insulin_per_WW

        merged_data.append([
            row['glucose'], row['insulin'], WW, hour, day_of_week,
            glucose_max - row['glucose'], glucose_min - row['glucose'], total_dose
        ])

# Tworzymy DataFrame z połączonych danych
data = pd.DataFrame(merged_data, columns=[
    'glucose', 'insulin', 'WW', 'hour', 'day_of_week',
    'glucose_diff_max', 'glucose_diff_min', 'total_dose'
])

# ✅ Osobny skaler dla insuliny
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

scaled_X = scaler_X.fit_transform(data.drop(columns=['total_dose']))
scaled_y = scaler_y.fit_transform(data[['total_dose']])

# ==== Przygotowanie sekwencji dla LSTM ====
sequence_length = 30
X, y = [], []
for i in range(len(scaled_X) - sequence_length):
    X.append(scaled_X[i:i + sequence_length])
    y.append(scaled_y[i + sequence_length])

X, y = np.array(X), np.array(y)

# ==== Definicja modelu LSTM ====
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(LSTM(128))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='relu'))

model.compile(optimizer='adam', loss='mse')

# ==== Trenowanie modelu ====
history = model.fit(X, y, epochs=30, batch_size=32)

# Zapis modelu
model.save('insulin_predictor.keras')

# ==== TESTY ====
test_input = np.array([[150, 5, 3, 9.5, 2 / 6, 20, -10]])
test_input_scaled = scaler_X.transform(test_input)
test_sequence = np.zeros((1, sequence_length, test_input.shape[1]))
test_sequence[0, -1, :] = test_input_scaled[0]

predicted_dose = model.predict(test_sequence)[0][0]
predicted_dose = scaler_y.inverse_transform([[predicted_dose]])[0][0]

# ✅ Korekta dawki
predicted_dose = max(0.5, min(predicted_dose, 15))

print(f"\n🔥 Proponowana dawka: {predicted_dose:.2f} jednostek")

if predicted_dose > test_input[0][1]:
    print("\n🔥 Dawka za mała → Zwiększ dawkę insuliny!")
elif predicted_dose < test_input[0][1]:
    print("\n❄️ Dawka za duża → Zmniejsz dawkę insuliny!")
else:
    print("\n✅ Dawka OK → Kontynuuj w tej samej dawce.")


Epoch 1/30


C:\Users\Micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 2.1296
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.6478
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.2102
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.1292
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1407
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1632
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1738
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1457
Epoch 9/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1724
Epoch 10/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1476
Epoch 11/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.1890
Epoch 12/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1619
Epoch 13/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1275
Epoch 14/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.1894
Epoch 15/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1505
Epoch 16/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/

C:\Users\Micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step

🔥 Proponowana dawka: 0.54 jednostek

❄️ Dawka za duża → Zmniejsz dawkę insuliny!


In [55]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.losses import Huber
from sklearn.preprocessing import StandardScaler
import json

# ==== Wczytywanie pliku CSV ====
glucose_data = pd.read_csv(
    'Michał._glucose_8-3-2025.csv',
    skiprows=1,
    delimiter=',',
    low_memory=False
)

# Konwersja daty na format datetime
glucose_data['Device Timestamp'] = pd.to_datetime(
    glucose_data['Device Timestamp'], 
    format='%d-%m-%Y %H:%M', 
    errors='coerce'
)

# Usuwamy błędne daty
glucose_data = glucose_data.dropna(subset=['Device Timestamp'])

# ✅ Łączymy dane z dwóch kolumn (Historic + Scan)
glucose_data['glucose'] = glucose_data['Historic Glucose mg/dL'].combine_first(
    glucose_data['Scan Glucose mg/dL']
)

glucose_data = glucose_data.rename(columns={'Device Timestamp': 'date'})
glucose_data = glucose_data[['date', 'glucose']]

# ==== Wczytywanie pliku JSON ====
with open('data (6).json', 'r') as f:
    meals_data = json.load(f)

# Przetwarzanie danych JSON
processed_data = []
for user_id, records in meals_data.items():
    for record in records:
        calculator = record.get('calculatorData')
        if calculator:
            glucose = calculator.get('glucose')
            insulin = calculator.get('units', {}).get('short')
            date = pd.to_datetime(calculator.get('date'), errors='coerce')

            carbs = sum(meal.get('carbs', 0) for meal in record.get('meals', []))

            processed_data.append([date, glucose, insulin, carbs])

# Tworzymy DataFrame z JSON
meals_df = pd.DataFrame(processed_data, columns=['date', 'glucose', 'insulin', 'carbs'])
meals_df['date'] = meals_df['date'].dt.tz_convert(None)

# ==== Sortowanie danych ====
glucose_data = glucose_data.sort_values('date').reset_index(drop=True)
meals_df = meals_df.sort_values('date').reset_index(drop=True)

# ==== Dopasowywanie danych z glukometru ====
merged_data = []
for index, row in meals_df.iterrows():
    meal_time = row['date']
    
    glucose_window = glucose_data[
        (glucose_data['date'] >= meal_time) & 
        (glucose_data['date'] <= meal_time + pd.Timedelta(hours=3))
    ]

    if not glucose_window.empty:
        glucose_values = glucose_window['glucose'].values.tolist()
        glucose_max = np.nanmax(glucose_values)
        glucose_min = np.nanmin(glucose_values)

        WW = row['carbs'] / 10  
        insulin_per_WW = row['insulin'] / WW if WW > 0 else 0

        hour = meal_time.hour + meal_time.minute / 60
        day_of_week = meal_time.dayofweek / 6  

        glucose_diff_max = glucose_max - row['glucose']
        glucose_diff_min = glucose_min - row['glucose']

        total_dose = WW * insulin_per_WW

        merged_data.append([
            row['glucose'], row['insulin'], WW, hour, day_of_week,
            glucose_diff_max, glucose_diff_min, total_dose
        ])

# Tworzymy DataFrame z połączonych danych
data = pd.DataFrame(merged_data, columns=[
    'glucose', 'insulin', 'WW', 'hour', 'day_of_week',
    'glucose_diff_max', 'glucose_diff_min', 'total_dose'
])

# ✅ Osobne skalery dla każdej zmiennej → Teraz `StandardScaler`
scalers = {}
for col in data.columns:
    scaler = StandardScaler()
    data[col] = scaler.fit_transform(data[[col]])
    scalers[col] = scaler

# ==== Przygotowanie sekwencji dla LSTM ====
sequence_length = 30
X, y = [], []
for i in range(len(data) - sequence_length):
    X.append(data.iloc[i:i + sequence_length].values)
    y.append(data['total_dose'].iloc[i + sequence_length])

X, y = np.array(X), np.array(y)

# ==== Definicja modelu LSTM ====
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(LSTM(128, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(LSTM(64))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(32, activation='swish'))
model.add(Dense(1, activation='swish'))

model.compile(optimizer='adam', loss=Huber())

# ==== Trenowanie modelu ====
history = model.fit(X, y, epochs=50, batch_size=16)

# Zapis modelu
model.save('insulin_predictor.keras')

# ==== TESTY ====
test_input = pd.DataFrame([[150, 5, 3, 9.5, 2/6, 20, -10, 0]],  # Dodajemy 0 dla 'total_dose'
                          columns=['glucose', 'insulin', 'WW', 'hour', 'day_of_week',
                                   'glucose_diff_max', 'glucose_diff_min', 'total_dose'])

# ✅ Skalowanie testów zgodnie z zakresem danych treningowych
for col in test_input.columns:
    test_input[col] = scalers[col].transform(test_input[[col]])

# ✅ Ustawiamy sekwencję testową o długości 30
test_sequence = np.zeros((1, sequence_length, test_input.shape[1]))
test_sequence[0, -1, :] = test_input.iloc[0].values

print(f"\n➡️ Kształt test_sequence: {test_sequence.shape}")

# ✅ Predykcja
predicted_dose = model.predict(test_sequence)[0][0]

# ✅ Inwersja skalowania na oryginalną wartość
predicted_dose = scalers['total_dose'].inverse_transform([[predicted_dose]])[0][0]

# ✅ Korekta dawki
predicted_dose = max(0.5, min(predicted_dose, 15))

# ✅ Wynik predykcji
print(f"\n🔥 Proponowana dawka: {predicted_dose:.2f} jednostek")

if predicted_dose > test_input['insulin'].iloc[0]:
    print("\n🔥 Dawka za mała → Zwiększ dawkę insuliny!")
elif predicted_dose < test_input['insulin'].iloc[0]:
    print("\n❄️ Dawka za duża → Zmniejsz dawkę insuliny!")
else:
    print("\n✅ Dawka OK → Kontynuuj w tej samej dawce.")


Epoch 1/50


C:\Users\Micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 0.6666
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.5298
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.4673
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.4050
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.5136
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.4587
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.3600
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.3919
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.4933
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.4009
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.3911
Epoch 12/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.4555
Epoch 13/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.4509
Epoch 14/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.4209
Epoch 15/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.4673
Epoch 16/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/

In [6]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.losses import Huber
from sklearn.preprocessing import StandardScaler
import json

# ==== Wczytywanie pliku CSV ====
glucose_data = pd.read_csv('Michał._glucose_8-3-2025.csv', skiprows=1, delimiter=',', low_memory=False)

# Konwersja daty na format datetime
glucose_data['Device Timestamp'] = pd.to_datetime(glucose_data['Device Timestamp'], format='%d-%m-%Y %H:%M', errors='coerce')

# Usuwamy błędne daty
glucose_data = glucose_data.dropna(subset=['Device Timestamp'])

# ✅ Łączymy dane z dwóch kolumn (Historic + Scan)
glucose_data['glucose'] = glucose_data['Historic Glucose mg/dL'].combine_first(glucose_data['Scan Glucose mg/dL'])

# Zmieniamy nazwę kolumny i wybieramy kluczowe kolumny
glucose_data = glucose_data.rename(columns={'Device Timestamp': 'date'})
glucose_data = glucose_data[['date', 'glucose']]

# ==== Wczytywanie pliku JSON ====
with open('data (6).json', 'r') as f:
    meals_data = json.load(f)

# Przetwarzanie danych JSON
processed_data = []
for user_id, records in meals_data.items():
    for record in records:
        calculator = record.get('calculatorData')
        if calculator:
            glucose = calculator.get('glucose')
            insulin = calculator.get('units', {}).get('short')
            date = pd.to_datetime(calculator.get('date'), errors='coerce')
            carbs = sum(meal.get('carbs', 0) for meal in record.get('meals', []))
            processed_data.append([date, glucose, insulin, carbs])

# Tworzymy DataFrame z JSON
meals_df = pd.DataFrame(processed_data, columns=['date', 'glucose', 'insulin', 'carbs'])
meals_df['date'] = meals_df['date'].dt.tz_convert(None)

# ==== Sortowanie danych ====
glucose_data = glucose_data.sort_values('date').reset_index(drop=True)
meals_df = meals_df.sort_values('date').reset_index(drop=True)

# ==== Dopasowywanie danych z glukometru ====
merged_data = []
for index, row in meals_df.iterrows():
    meal_time = row['date']
    glucose_window = glucose_data[(glucose_data['date'] >= meal_time) & (glucose_data['date'] <= meal_time + pd.Timedelta(hours=3))]

    if not glucose_window.empty:
        glucose_values = glucose_window['glucose'].values.tolist()
        glucose_max = np.nanmax(glucose_values)
        glucose_min = np.nanmin(glucose_values)

        WW = row['carbs'] / 10
        insulin_per_WW = row['insulin'] / WW if WW > 0 else 0

        hour = meal_time.hour + meal_time.minute / 60
        glucose_diff_max = glucose_max - row['glucose']
        glucose_diff_min = glucose_min - row['glucose']

        total_dose = WW * insulin_per_WW

        merged_data.append([
            row['glucose'], row['insulin'], WW, hour,
            glucose_diff_max, glucose_diff_min, total_dose
        ])

# Tworzymy DataFrame z połączonych danych
data = pd.DataFrame(merged_data, columns=[
    'glucose', 'insulin', 'WW', 'hour',
    'glucose_diff_max', 'glucose_diff_min', 'total_dose'
])

# ✅ Osobne skalery dla każdej zmiennej → StandardScaler
scalers = {}
for col in data.columns:
    scaler = StandardScaler()
    data[col] = scaler.fit_transform(data[[col]])
    scalers[col] = scaler

# ==== Przygotowanie sekwencji dla LSTM ====
sequence_length = 30
X, y = [], []
for i in range(len(data) - sequence_length):
    X.append(data.iloc[i:i + sequence_length].values)
    y.append(data['total_dose'].iloc[i + sequence_length])

X, y = np.array(X), np.array(y)

# ==== Definicja modelu LSTM ====
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(LSTM(256, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(LSTM(128))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss=Huber())

# ==== Trenowanie modelu ====
history = model.fit(X, y, epochs=100, batch_size=16)

# Zapis modelu
model.save('insulin_predictor.keras')

# ==== TESTY ====
test_input = pd.DataFrame([[150, 5, 3, 9.5, 20, -10, 0]], columns=data.columns)

# ✅ Skalowanie testów zgodnie z zakresem danych treningowych
for col in test_input.columns:
    test_input[col] = scalers[col].transform(test_input[[col]])

# ✅ Ustawiamy sekwencję testową o długości 30
test_sequence = np.zeros((1, sequence_length, test_input.shape[1]))
test_sequence[0, -1, :] = test_input.iloc[0].values

# ✅ Predykcja
predicted_dose = model.predict(test_sequence)[0][0]
predicted_dose = scalers['total_dose'].inverse_transform([[predicted_dose]])[0][0]
predicted_dose = max(0.5, min(predicted_dose, 15))

# ✅ Wynik predykcji
print(f"\n🔥 Proponowana dawka: {predicted_dose:.2f} jednostek")

if predicted_dose > 5:
    print("\n🔥 Dawka za mała → Zwiększ dawkę insuliny!")
elif predicted_dose < 5:
    print("\n❄️ Dawka za duża → Zmniejsz dawkę insuliny!")
else:
    print("\n✅ Dawka OK → Kontynuuj w tej samej dawce.")


Epoch 1/100


C:\Users\Micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - loss: 0.8113
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.6757
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.7432
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.5914
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.5775
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.5472
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.5136
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.4675
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.6370
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.5143
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.4196
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.6058
Epoch 13/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.5368
Epoch 14/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.6561
Epoch 15/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.5115
Epoch 16/100
4/4 ━━━━━━━━━━━━━━

In [59]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.losses import Huber
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
import json

# ==== Wczytywanie pliku CSV ====
glucose_data = pd.read_csv('Michał._glucose_8-3-2025.csv', skiprows=1, delimiter=',', low_memory=False)

# Konwersja daty na format datetime
glucose_data['Device Timestamp'] = pd.to_datetime(glucose_data['Device Timestamp'], format='%d-%m-%Y %H:%M', errors='coerce')

# Usuwamy błędne daty
glucose_data = glucose_data.dropna(subset=['Device Timestamp'])

# ✅ Łączymy dane z dwóch kolumn (Historic + Scan)
glucose_data['glucose'] = glucose_data['Historic Glucose mg/dL'].combine_first(glucose_data['Scan Glucose mg/dL'])

# Zmieniamy nazwę kolumny i wybieramy kluczowe kolumny
glucose_data = glucose_data.rename(columns={'Device Timestamp': 'date'})
glucose_data = glucose_data[['date', 'glucose']]

# ==== Wczytywanie pliku JSON ====
with open('data (6).json', 'r') as f:
    meals_data = json.load(f)

# Przetwarzanie danych JSON
processed_data = []
for user_id, records in meals_data.items():
    for record in records:
        calculator = record.get('calculatorData')
        if calculator:
            glucose = calculator.get('glucose')
            insulin = calculator.get('units', {}).get('short')
            date = pd.to_datetime(calculator.get('date'), errors='coerce')
            carbs = sum(meal.get('carbs', 0) for meal in record.get('meals', []))
            processed_data.append([date, glucose, insulin, carbs])

# Tworzymy DataFrame z JSON
meals_df = pd.DataFrame(processed_data, columns=['date', 'glucose', 'insulin', 'carbs'])
meals_df['date'] = meals_df['date'].dt.tz_convert(None)

# ==== Sortowanie danych ====
glucose_data = glucose_data.sort_values('date').reset_index(drop=True)
meals_df = meals_df.sort_values('date').reset_index(drop=True)

# ==== Dopasowywanie danych z glukometru ====
merged_data = []
for index, row in meals_df.iterrows():
    meal_time = row['date']
    glucose_window = glucose_data[(glucose_data['date'] >= meal_time) & (glucose_data['date'] <= meal_time + pd.Timedelta(hours=3))]

    if not glucose_window.empty:
        glucose_values = glucose_window['glucose'].values.tolist()
        glucose_max = np.nanmax(glucose_values)
        glucose_min = np.nanmin(glucose_values)

        WW = row['carbs'] / 10
        insulin_per_WW = row['insulin'] / WW if WW > 0 else 0

        hour = meal_time.hour + meal_time.minute / 60
        glucose_diff_max = glucose_max - row['glucose']
        glucose_diff_min = glucose_min - row['glucose']

        total_dose = WW * insulin_per_WW

        merged_data.append([
            row['glucose'], row['insulin'], WW, hour,
            glucose_diff_max, glucose_diff_min, total_dose
        ])

# Tworzymy DataFrame z połączonych danych
data = pd.DataFrame(merged_data, columns=[
    'glucose', 'insulin', 'WW', 'hour',
    'glucose_diff_max', 'glucose_diff_min', 'total_dose'
])

# ✅ Osobne skalery dla każdej zmiennej → StandardScaler
scalers = {}
for col in data.columns:
    scaler = StandardScaler()
    data[col] = scaler.fit_transform(data[[col]])
    scalers[col] = scaler

# ==== Przygotowanie sekwencji dla GRU ====
sequence_length = 20
X, y = [], []
for i in range(len(data) - sequence_length):
    X.append(data.iloc[i:i + sequence_length].values)
    y.append(data['total_dose'].iloc[i + sequence_length])

X, y = np.array(X), np.array(y)

# ==== Definicja modelu GRU ====
model = Sequential()
model.add(GRU(128, return_sequences=True, input_shape=(X.shape[1], X.shape[2]), kernel_regularizer=l2(0.0001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(GRU(64, kernel_regularizer=l2(0.0001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='relu'))

# ✅ Kompilacja modelu
model.compile(optimizer='adam', loss=Huber())

# ✅ Early stopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# ==== Trenowanie modelu ====
history = model.fit(X, y, epochs=100, batch_size=16, callbacks=[early_stopping])

# Zapis modelu
model.save('insulin_predictor.keras')

# ==== TESTY ====
test_input = pd.DataFrame([[150, 5, 5, 9.5, 20, -10, 0]], columns=data.columns)

# ✅ Skalowanie testów zgodnie z zakresem danych treningowych
for col in test_input.columns:
    test_input[col] = scalers[col].transform(test_input[[col]])

# ✅ Ustawiamy sekwencję testową o długości 20
test_sequence = np.zeros((1, sequence_length, test_input.shape[1]))
test_sequence[0, -1, :] = test_input.iloc[0].values

# ✅ Predykcja
predicted_dose = model.predict(test_sequence)[0][0]
predicted_dose = scalers['total_dose'].inverse_transform(np.array([[predicted_dose]]))[0][0]
predicted_dose = max(0.5, min(predicted_dose, 15))

# ✅ Wynik predykcji
print(f"\n🔥 Proponowana dawka: {predicted_dose:.2f} jednostek")

if predicted_dose > 5:
    print("\n🔥 Dawka za mała → Zwiększ dawkę insuliny!")
elif predicted_dose < 5:
    print("\n❄️ Dawka za duża → Zmniejsz dawkę insuliny!")
else:
    print("\n✅ Dawka OK → Kontynuuj w tej samej dawce.")


Epoch 1/100


C:\Users\Micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.6059
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.6081
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.5289
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.4871
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.3482
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.5216
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.4381
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.4710
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.3701
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.3854
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.3852
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.4098
Epoch 13/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.3266
Epoch 14/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.4025
Epoch 15/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3966 
Epoch 16/100
5/5 ━━━━━━━━━━━━━━

In [65]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import Huber
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
import json

# ==== Wczytywanie pliku CSV ====
glucose_data = pd.read_csv('Michał._glucose_8-3-2025.csv', skiprows=1, delimiter=',', low_memory=False)

# Konwersja daty na format datetime
glucose_data['Device Timestamp'] = pd.to_datetime(glucose_data['Device Timestamp'], format='%d-%m-%Y %H:%M', errors='coerce')

# Usuwamy błędne daty
glucose_data = glucose_data.dropna(subset=['Device Timestamp'])

# ✅ Łączymy dane z dwóch kolumn (Historic + Scan)
glucose_data['glucose'] = glucose_data['Historic Glucose mg/dL'].combine_first(glucose_data['Scan Glucose mg/dL'])

# Zmieniamy nazwę kolumny i wybieramy kluczowe kolumny
glucose_data = glucose_data.rename(columns={'Device Timestamp': 'date'})
glucose_data = glucose_data[['date', 'glucose']]

# ==== Wczytywanie pliku JSON ====
with open('data (6).json', 'r') as f:
    meals_data = json.load(f)

# Przetwarzanie danych JSON
processed_data = []
for user_id, records in meals_data.items():
    for record in records:
        calculator = record.get('calculatorData')
        if calculator:
            glucose = calculator.get('glucose')
            insulin = calculator.get('units', {}).get('short')
            date = pd.to_datetime(calculator.get('date'), errors='coerce')
            carbs = sum(meal.get('carbs', 0) for meal in record.get('meals', []))
            processed_data.append([date, glucose, insulin, carbs])

# Tworzymy DataFrame z JSON
meals_df = pd.DataFrame(processed_data, columns=['date', 'glucose', 'insulin', 'carbs'])
meals_df['date'] = meals_df['date'].dt.tz_convert(None)

# ==== Sortowanie danych ====
glucose_data = glucose_data.sort_values('date').reset_index(drop=True)
meals_df = meals_df.sort_values('date').reset_index(drop=True)

# ==== Dopasowywanie danych z glukometru ====
merged_data = []
for index, row in meals_df.iterrows():
    meal_time = row['date']
    glucose_window = glucose_data[(glucose_data['date'] >= meal_time) & (glucose_data['date'] <= meal_time + pd.Timedelta(hours=3))]

    if not glucose_window.empty:
        glucose_values = glucose_window['glucose'].values.tolist()
        glucose_max = np.nanmax(glucose_values)
        glucose_min = np.nanmin(glucose_values)

        WW = row['carbs'] / 10
        insulin_per_WW = row['insulin'] / WW if WW > 0 else 0

        hour = meal_time.hour + meal_time.minute / 60
        glucose_diff_max = glucose_max - row['glucose']
        glucose_diff_min = glucose_min - row['glucose']

        # ✅ Zwiększamy wagę dla WW
        total_dose = WW * insulin_per_WW * 1.5

        merged_data.append([
            row['glucose'], row['insulin'], WW, hour,
            glucose_diff_max, glucose_diff_min, total_dose
        ])

# Tworzymy DataFrame z połączonych danych
data = pd.DataFrame(merged_data, columns=[
    'glucose', 'insulin', 'WW', 'hour',
    'glucose_diff_max', 'glucose_diff_min', 'total_dose'
])

# ✅ Osobne skalery dla każdej zmiennej → StandardScaler
scalers = {}
for col in data.columns:
    scaler = StandardScaler()
    data[col] = scaler.fit_transform(data[[col]])
    scalers[col] = scaler

# ==== Przygotowanie sekwencji dla GRU ====
sequence_length = 20
X, y = [], []
for i in range(len(data) - sequence_length):
    X.append(data.iloc[i:i + sequence_length].values)
    y.append(data['total_dose'].iloc[i + sequence_length])

X, y = np.array(X), np.array(y)

# ==== Definicja modelu GRU ====
model = Sequential()
model.add(GRU(128, return_sequences=True, input_shape=(X.shape[1], X.shape[2]), kernel_regularizer=l2(0.0001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(GRU(64, kernel_regularizer=l2(0.0001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='relu'))

# ✅ Kompilacja modelu
model.compile(optimizer='adam', loss=Huber())

# ✅ Early stopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# ==== Trenowanie modelu ====
history = model.fit(X, y, epochs=100, batch_size=16, callbacks=[early_stopping])

# Zapis modelu
model.save('insulin_predictor.keras')

# ==== TESTY ====
test_input = pd.DataFrame([[150, 5, 3, 9.5, 20, -10, 0]], columns=data.columns)

# ✅ Skalowanie testów zgodnie z zakresem danych treningowych
for col in test_input.columns:
    test_input[col] = scalers[col].transform(test_input[[col]])

# ✅ Ustawiamy sekwencję testową o długości 20
test_sequence = np.zeros((1, sequence_length, test_input.shape[1]))
test_sequence[0, -1, :] = test_input.iloc[0].values

# ✅ Predykcja
predicted_dose = model.predict(test_sequence)[0][0]
predicted_dose = scalers['total_dose'].inverse_transform([[predicted_dose]])[0][0]
predicted_dose = max(0.5, min(predicted_dose, 15))

# ✅ Wynik predykcji
print(f"\n🔥 Proponowana dawka: {predicted_dose:.2f} jednostek")

if predicted_dose > 5:
    print("\n🔥 Dawka za mała → Zwiększ dawkę insuliny!")
elif predicted_dose < 5:
    print("\n❄️ Dawka za duża → Zmniejsz dawkę insuliny!")
else:
    print("\n✅ Dawka OK → Kontynuuj w tej samej dawce.")


Epoch 1/100


C:\Users\Micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.7470
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.6925 
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.5648 
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5052 
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4516 
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4365 
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4290 
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4336 
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4897 
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4089 
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3406 
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4082 
Epoch 13/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3555 
Epoch 14/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3860 
Epoch 15/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4150 
Epoch 16/100
5/5 ━━━━━━━━━━━━━━━

In [71]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import Huber
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
import json

# ==== Wczytywanie pliku CSV ====
glucose_data = pd.read_csv('Michał._glucose_8-3-2025.csv', skiprows=1, delimiter=',', low_memory=False)

glucose_data['Device Timestamp'] = pd.to_datetime(glucose_data['Device Timestamp'], format='%d-%m-%Y %H:%M', errors='coerce')
glucose_data = glucose_data.dropna(subset=['Device Timestamp'])
glucose_data['glucose'] = glucose_data['Historic Glucose mg/dL'].combine_first(glucose_data['Scan Glucose mg/dL'])
glucose_data = glucose_data.rename(columns={'Device Timestamp': 'date'})
glucose_data = glucose_data[['date', 'glucose']]

# ==== Wczytywanie pliku JSON ====
with open('data (6).json', 'r') as f:
    meals_data = json.load(f)

processed_data = []
for user_id, records in meals_data.items():
    for record in records:
        calculator = record.get('calculatorData')
        if calculator:
            glucose = calculator.get('glucose')
            insulin = calculator.get('units', {}).get('short')
            date = pd.to_datetime(calculator.get('date'), errors='coerce')
            carbs = sum(meal.get('carbs', 0) for meal in record.get('meals', []))
            processed_data.append([date, glucose, insulin, carbs])

meals_df = pd.DataFrame(processed_data, columns=['date', 'glucose', 'insulin', 'carbs'])
meals_df['date'] = meals_df['date'].dt.tz_convert(None)

# ==== Dopasowywanie danych ====
glucose_data = glucose_data.sort_values('date').reset_index(drop=True)
meals_df = meals_df.sort_values('date').reset_index(drop=True)

merged_data = []
for index, row in meals_df.iterrows():
    meal_time = row['date']
    glucose_window = glucose_data[(glucose_data['date'] >= meal_time + pd.Timedelta(hours=1)) & (glucose_data['date'] <= meal_time + pd.Timedelta(hours=2.5))]

    if not glucose_window.empty:
        glucose_max = np.nanmax(glucose_window['glucose'].values)
        glucose_min = np.nanmin(glucose_window['glucose'].values)

        WW = row['carbs'] / 10
        insulin_per_WW = row['insulin'] / WW if WW > 0 else 0

        hour = meal_time.hour + meal_time.minute / 60
        glucose_diff_max = glucose_max - row['glucose']
        glucose_diff_min = glucose_min - row['glucose']

        total_dose = WW * insulin_per_WW * 1.2  # Zwiększamy wpływ WW

        merged_data.append([
            row['glucose'], row['insulin'], WW, hour,
            glucose_diff_max, glucose_diff_min, total_dose
        ])

# Tworzymy DataFrame z połączonych danych
data = pd.DataFrame(merged_data, columns=[
    'glucose', 'insulin', 'WW', 'hour',
    'glucose_diff_max', 'glucose_diff_min', 'total_dose'
])

# ✅ Osobne skalery dla każdej zmiennej
scalers = {}
for col in data.columns:
    scaler = StandardScaler()
    data[col] = scaler.fit_transform(data[[col]])
    scalers[col] = scaler

# ==== Przygotowanie sekwencji dla GRU ====
sequence_length = 20
X, y = [], []
for i in range(len(data) - sequence_length):
    X.append(data.iloc[i:i + sequence_length].values)
    y.append(data['total_dose'].iloc[i + sequence_length])

X, y = np.array(X), np.array(y)

# ==== Definicja modelu GRU ====
model = Sequential()
model.add(GRU(256, return_sequences=True, input_shape=(X.shape[1], X.shape[2]), kernel_regularizer=l2(0.0001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(GRU(128, return_sequences=True, kernel_regularizer=l2(0.0001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(GRU(64, kernel_regularizer=l2(0.0001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(32, activation='swish'))
model.add(Dense(1, activation='relu'))

model.compile(optimizer='adam', loss=Huber())

# ✅ Early stopping
early_stopping = EarlyStopping(monitor='loss', patience=15, restore_best_weights=True)

# ==== Trenowanie modelu ====
history = model.fit(X, y, epochs=150, batch_size=16, callbacks=[early_stopping])

# Zapis modelu
model.save('insulin_predictor.keras')

# ==== TESTY ====
test_input = pd.DataFrame([[150, 5, 5, 9.5, 20, -10, 0]], columns=data.columns)

# ✅ Skalowanie testów
for col in test_input.columns:
    test_input[col] = scalers[col].transform(test_input[[col]])

# ✅ Ustawiamy sekwencję testową o długości 20
test_sequence = np.zeros((1, sequence_length, test_input.shape[1]))
test_sequence[0, -1, :] = test_input.iloc[0].values

# ✅ Debug inputu
print("\n➡️ Test input (scaled):", test_sequence[0, -1, :])

# ✅ Predykcja
predicted_dose = model.predict(test_sequence)[0][0]
predicted_dose = scalers['total_dose'].inverse_transform([[predicted_dose]])[0][0]
predicted_dose = max(0.5, min(predicted_dose, 15))

# ✅ Wynik predykcji
print(f"\n🔥 Proponowana dawka: {predicted_dose:.2f} jednostek")

if predicted_dose > 5:
    print("\n🔥 Dawka za mała → Zwiększ dawkę insuliny!")
elif predicted_dose < 5:
    print("\n❄️ Dawka za duża → Zmniejsz dawkę insuliny!")
else:
    print("\n✅ Dawka OK → Kontynuuj w tej samej dawce.")


test_input = pd.DataFrame([[150, 3, 3, 9.5, 20, -10, 0]], columns=data.columns)

# ✅ Skalowanie testów
for col in test_input.columns:
    test_input[col] = scalers[col].transform(test_input[[col]])

# ✅ Ustawiamy sekwencję testową o długości 20
test_sequence = np.zeros((1, sequence_length, test_input.shape[1]))
test_sequence[0, -1, :] = test_input.iloc[0].values

# ✅ Debug inputu
print("\n➡️ Test input (scaled):", test_sequence[0, -1, :])

# ✅ Predykcja
predicted_dose = model.predict(test_sequence)[0][0]
predicted_dose = scalers['total_dose'].inverse_transform([[predicted_dose]])[0][0]
predicted_dose = max(0.5, min(predicted_dose, 15))

# ✅ Wynik predykcji
print(f"\n🔥 Proponowana dawka: {predicted_dose:.2f} jednostek")

if predicted_dose > 5:
    print("\n🔥 Dawka za mała → Zwiększ dawkę insuliny!")
elif predicted_dose < 5:
    print("\n❄️ Dawka za duża → Zmniejsz dawkę insuliny!")
else:
    print("\n✅ Dawka OK → Kontynuuj w tej samej dawce.")


Epoch 1/150


C:\Users\Micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.6255
Epoch 2/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.5194
Epoch 3/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.5103
Epoch 4/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.4898
Epoch 5/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.4554
Epoch 6/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.4719
Epoch 7/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.4923
Epoch 8/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.4308
Epoch 9/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.4256
Epoch 10/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.4565
Epoch 11/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.4960
Epoch 12/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.4608
Epoch 13/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.4458
Epoch 14/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.4554
Epoch 15/150
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.4756
Epoch 16/150
5/5 ━━━━━━━━━━━━━━

In [72]:
# ==== TESTY ====
test_input = pd.DataFrame([[150, 5, 5, 9.5, 20, -10, 0]], columns=data.columns)

# ✅ Skalowanie testów
for col in test_input.columns:
    test_input[col] = scalers[col].transform(test_input[[col]])

# ✅ Ustawiamy sekwencję testową o długości 20
test_sequence = np.zeros((1, sequence_length, test_input.shape[1]))
test_sequence[0, -1, :] = test_input.iloc[0].values

# ✅ Debug inputu
print("\n➡️ Test input (scaled):", test_sequence[0, -1, :])

# ✅ Predykcja
predicted_dose = model.predict(test_sequence)[0][0]
predicted_dose = scalers['total_dose'].inverse_transform([[predicted_dose]])[0][0]
predicted_dose = max(0.5, min(predicted_dose, 15))

# ✅ Wynik predykcji
print(f"\n🔥 Proponowana dawka: {predicted_dose:.2f} jednostek")

if predicted_dose > 5:
    print("\n🔥 Dawka za mała → Zwiększ dawkę insuliny!")
elif predicted_dose < 5:
    print("\n❄️ Dawka za duża → Zmniejsz dawkę insuliny!")
else:
    print("\n✅ Dawka OK → Kontynuuj w tej samej dawce.")


test_input = pd.DataFrame([[150, 3, 3, 9.5, 20, -10, 0]], columns=data.columns)

# ✅ Skalowanie testów
for col in test_input.columns:
    test_input[col] = scalers[col].transform(test_input[[col]])

# ✅ Ustawiamy sekwencję testową o długości 20
test_sequence = np.zeros((1, sequence_length, test_input.shape[1]))
test_sequence[0, -1, :] = test_input.iloc[0].values

# ✅ Debug inputu
print("\n➡️ Test input (scaled):", test_sequence[0, -1, :])

# ✅ Predykcja
predicted_dose = model.predict(test_sequence)[0][0]
predicted_dose = scalers['total_dose'].inverse_transform([[predicted_dose]])[0][0]
predicted_dose = max(0.5, min(predicted_dose, 15))

# ✅ Wynik predykcji
print(f"\n🔥 Proponowana dawka: {predicted_dose:.2f} jednostek")

if predicted_dose > 5:
    print("\n🔥 Dawka za mała → Zwiększ dawkę insuliny!")
elif predicted_dose < 5:
    print("\n❄️ Dawka za duża → Zmniejsz dawkę insuliny!")
else:
    print("\n✅ Dawka OK → Kontynuuj w tej samej dawce.")


➡️ Test input (scaled): [ 0.44611978  0.49302543  0.5981197  -0.96110206 -0.59144429  0.31369237
 -1.75641933]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step

🔥 Proponowana dawka: 4.16 jednostek

❄️ Dawka za duża → Zmniejsz dawkę insuliny!

➡️ Test input (scaled): [ 0.44611978 -0.26228289 -0.27595919 -0.96110206 -0.59144429  0.31369237
 -1.75641933]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step

🔥 Proponowana dawka: 4.26 jednostek

❄️ Dawka za duża → Zmniejsz dawkę insuliny!


In [70]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import Huber
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
import json

# ==== Wczytywanie pliku CSV ====
glucose_data = pd.read_csv('Michał._glucose_8-3-2025.csv', skiprows=1, delimiter=',', low_memory=False)

# Konwersja daty na format datetime
glucose_data['Device Timestamp'] = pd.to_datetime(glucose_data['Device Timestamp'], format='%d-%m-%Y %H:%M', errors='coerce')

# Usuwamy błędne daty
glucose_data = glucose_data.dropna(subset=['Device Timestamp'])

# ✅ Łączymy dane z dwóch kolumn (Historic + Scan)
glucose_data['glucose'] = glucose_data['Historic Glucose mg/dL'].combine_first(glucose_data['Scan Glucose mg/dL'])

# Zmieniamy nazwę kolumny i wybieramy kluczowe kolumny
glucose_data = glucose_data.rename(columns={'Device Timestamp': 'date'})
glucose_data = glucose_data[['date', 'glucose']]

# ==== Wczytywanie pliku JSON ====
with open('data (6).json', 'r') as f:
    meals_data = json.load(f)

# Przetwarzanie danych JSON
processed_data = []
for user_id, records in meals_data.items():
    for record in records:
        calculator = record.get('calculatorData')
        if calculator:
            glucose = calculator.get('glucose')
            insulin = calculator.get('units', {}).get('short')
            date = pd.to_datetime(calculator.get('date'), errors='coerce')
            carbs = sum(meal.get('carbs', 0) for meal in record.get('meals', []))
            processed_data.append([date, glucose, insulin, carbs])

# Tworzymy DataFrame z JSON
meals_df = pd.DataFrame(processed_data, columns=['date', 'glucose', 'insulin', 'carbs'])
meals_df['date'] = meals_df['date'].dt.tz_convert(None)

# ==== Dopasowywanie danych ====
glucose_data = glucose_data.sort_values('date').reset_index(drop=True)
meals_df = meals_df.sort_values('date').reset_index(drop=True)

merged_data = []
for index, row in meals_df.iterrows():
    meal_time = row['date']
    glucose_window = glucose_data[(glucose_data['date'] >= meal_time + pd.Timedelta(hours=1)) & (glucose_data['date'] <= meal_time + pd.Timedelta(hours=2.5))]

    if not glucose_window.empty:
        glucose_max = np.nanmax(glucose_window['glucose'].values)
        glucose_min = np.nanmin(glucose_window['glucose'].values)

        WW = row['carbs'] / 10
        insulin_per_WW = row['insulin'] / WW if WW > 0 else 0

        hour = meal_time.hour + meal_time.minute / 60
        glucose_diff_max = glucose_max - row['glucose']
        glucose_diff_min = glucose_min - row['glucose']

        total_dose = row['insulin']

        merged_data.append([
            row['glucose'], row['insulin'], WW, hour,
            glucose_diff_max, glucose_diff_min, total_dose
        ])

# Tworzymy DataFrame z połączonych danych
data = pd.DataFrame(merged_data, columns=[
    'glucose', 'insulin', 'WW', 'hour',
    'glucose_diff_max', 'glucose_diff_min', 'total_dose'
])

# ✅ Osobne skalery dla każdej zmiennej → StandardScaler
scalers = {}
for col in data.columns:
    scaler = StandardScaler()
    data[col] = scaler.fit_transform(data[[col]])
    scalers[col] = scaler

# ==== Przygotowanie sekwencji dla GRU ====
sequence_length = 20
X, y = [], []
for i in range(len(data) - sequence_length):
    X.append(data.iloc[i:i + sequence_length].values)
    y.append(data['total_dose'].iloc[i + sequence_length])

X, y = np.array(X), np.array(y)

# ==== Definicja modelu GRU ====
model = Sequential()
model.add(GRU(128, return_sequences=True, input_shape=(X.shape[1], X.shape[2]), kernel_regularizer=l2(0.0001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(GRU(64, kernel_regularizer=l2(0.0001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='relu'))

# ✅ Kompilacja modelu
model.compile(optimizer='adam', loss=Huber())

# ✅ Early stopping
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# ==== Trenowanie modelu ====
history = model.fit(X, y, epochs=100, batch_size=16, callbacks=[early_stopping])

# Zapis modelu
model.save('insulin_predictor.keras')

# ==== TESTY ====
test_input = pd.DataFrame([[150, 5, 5, 9.5, 20, -10, 0]], columns=data.columns)

# ✅ Skalowanie testów zgodnie z zakresem danych treningowych
test_input_scaled = np.array([scalers[col].transform(test_input[[col]])[0][0] for col in test_input.columns])

# ✅ Ustawiamy sekwencję testową o długości 20
test_sequence = np.zeros((1, sequence_length, test_input_scaled.shape[0]))
test_sequence[0, -1, :] = test_input_scaled

# ✅ Predykcja
predicted_dose = model.predict(test_sequence)[0][0]
predicted_dose = scalers['total_dose'].inverse_transform([[predicted_dose]])[0][0]
predicted_dose = max(0.5, min(predicted_dose, 15))

# ✅ Wynik predykcji
print(f"\n🔥 Proponowana dawka: {predicted_dose:.2f} jednostek")

if predicted_dose > 5:
    print("\n🔥 Dawka za mała → Zwiększ dawkę insuliny!")
elif predicted_dose < 5:
    print("\n❄️ Dawka za duża → Zmniejsz dawkę insuliny!")
else:
    print("\n✅ Dawka OK → Kontynuuj w tej samej dawce.")


Epoch 1/100


C:\Users\Micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.4221
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4016 
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3565 
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3120 
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4140 
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4127 
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.4129 
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3440 
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.4405 
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3300 
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2984 
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3763 
Epoch 13/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3115 
Epoch 14/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.3654 
Epoch 15/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.3655 
Epoch 16/100
5/5 ━━━━━━━━━━━━━━